In [0]:
!git clone https://github.com/MaurizioFD/RecSys_Course_2018.git
!mv -v ./RecSys_Course_2018/* ./
!rm -r ./RecSys_Course_2018/
!pip install -r requirements.txt
!python run_compile_all_cython.py

In [0]:
URM_file = open("data/data_train.csv", 'r')

lines = URM_file.readline()

def rowSplit (rowString):
    split = rowString.split(",")
    split[2] = split[2].replace("\n", " ")

    split[0] = int(split[0])
    split[1] = int(split[1])
    split[2] = float(split[2])

    result = tuple(split)

    return result


URM_file.seek(0)
URM_tuples = []

for line in URM_file:
   URM_tuples.append(rowSplit(line))

userList, itemList, ratingList = zip(*URM_tuples)

userList = list(userList)
itemList = list(itemList)
ratingList = list(ratingList)

userList_unique = list(set(userList))
itemList_unique = list(set(itemList))
numUsers = len(userList_unique)
numItems = len(itemList_unique)

import scipy.sparse as sps
URM_all = sps.coo_matrix((ratingList, (userList, itemList)))
URM_all = URM_all.tocsr()

import numpy as np
import scipy.sparse as sps

def train_test_holdout(URM_all, train_perc = 0.8):


    numInteractions = URM_all.nnz
    URM_all = URM_all.tocoo()
    shape = URM_all.shape


    train_mask = np.random.choice([True,False], numInteractions, p=[train_perc, 1-train_perc])


    URM_train = sps.coo_matrix((URM_all.data[train_mask], (URM_all.row[train_mask], URM_all.col[train_mask])), shape=shape)
    URM_train = URM_train.tocsr()

    test_mask = np.logical_not(train_mask)

    URM_test = sps.coo_matrix((URM_all.data[test_mask], (URM_all.row[test_mask], URM_all.col[test_mask])), shape=shape)
    URM_test = URM_test.tocsr()

    return URM_train, URM_test

URM_train, URM_test = train_test_holdout(URM_all, train_perc=0.8)

In [0]:
from ParameterTuning.run_parameter_search import *
from Base.Evaluation import *
from Base import *


In [0]:
collaborative_algorithm_list = [
        Random,
        TopPop,
        P3alphaRecommender,
        RP3betaRecommender,
        ItemKNNCFRecommender,
        UserKNNCFRecommender,
        MatrixFactorization_BPR_Cython,
        MatrixFactorization_FunkSVD_Cython,
        PureSVDRecommender,
        SLIM_BPR_Cython,
        SLIMElasticNetRecommender
    ]

In [0]:
from Base.Evaluation.Evaluator import EvaluatorHoldout

evaluator_validation = EvaluatorHoldout(URM_test, cutoff_list=[10])

In [0]:
runParameterSearch_Collaborative(MatrixFactorization_BPR_Cython,
                                 URM_train=URM_train,
                                 n_cases = 8,
                                 metric_to_optimize = "MAP",
                                 evaluator_validation_earlystopping = evaluator_validation,
                                 evaluator_validation = evaluator_validation
                                 )

Iteration No: 1 started. Evaluating function at random point.
MatrixFactorization_BPR_Cython_Recommender: URM Detected 4316 (13.96 %) cold users.
MatrixFactorization_BPR_Cython_Recommender: URM Detected 3679 (19.89 %) cold items.
SearchBayesianSkopt: Testing config: {'sgd_mode': 'adam', 'epochs': 1500, 'num_factors': 18, 'batch_size': 32, 'positive_reg': 4.327706618859571e-05, 'negative_reg': 0.001081034198281375, 'learning_rate': 0.009072343106629016}
MF_BPR: Processed 30880 ( 99.90% ) in 1.02 seconds. BPR loss 6.17E-03. Sample per second: 30142
MF_BPR: Epoch 1 of 1500. Elapsed time 0.13 sec
MF_BPR: Processed 30880 ( 99.90% ) in 0.16 seconds. BPR loss 3.28E-02. Sample per second: 189054
MF_BPR: Epoch 2 of 1500. Elapsed time 0.27 sec
MF_BPR: Processed 30880 ( 99.90% ) in 0.30 seconds. BPR loss 1.45E-01. Sample per second: 102153
MF_BPR: Epoch 3 of 1500. Elapsed time 0.41 sec
MF_BPR: Processed 30880 ( 99.90% ) in 0.44 seconds. BPR loss 4.26E-01. Sample per second: 70007
MF_BPR: Epoch 4 

/usr/local/lib/python3.6/dist-packages/skopt/optimizer/optimizer.py:399: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


MF_BPR: Processed 30910 ( 99.99% ) in 1.43 seconds. BPR loss 4.23E-02. Sample per second: 21683
MF_BPR: Epoch 1 of 1500. Elapsed time 0.64 sec
MF_BPR: Processed 30910 ( 99.99% ) in 1.08 seconds. BPR loss 4.86E-02. Sample per second: 28699
MF_BPR: Epoch 2 of 1500. Elapsed time 1.29 sec
MF_BPR: Processed 30910 ( 99.99% ) in 0.73 seconds. BPR loss 5.78E-02. Sample per second: 42305
MF_BPR: Epoch 3 of 1500. Elapsed time 1.95 sec
MF_BPR: Processed 30910 ( 99.99% ) in 1.38 seconds. BPR loss 6.99E-02. Sample per second: 22377
MF_BPR: Epoch 4 of 1500. Elapsed time 2.60 sec
MF_BPR: Processed 30910 ( 99.99% ) in 1.03 seconds. BPR loss 8.82E-02. Sample per second: 29980
MF_BPR: Validation begins...
EvaluatorHoldout: Processed 20381 ( 100.00% ) in 17.30 sec. Users per second: 1178
MF_BPR: CUTOFF: 10 - ROC_AUC: 0.0014774, PRECISION: 0.0002944, PRECISION_RECALL_MIN_DEN: 0.0007837, RECALL: 0.0007584, MAP: 0.0002313, MRR: 0.0008418, NDCG: 0.0004939, F1: 0.0004242, HIT_RATE: 0.0029439, ARHR: 0.0008418,

/usr/local/lib/python3.6/dist-packages/skopt/optimizer/optimizer.py:399: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


MF_BPR: Processed 30910 ( 99.99% ) in 1.08 seconds. BPR loss 4.14E-02. Sample per second: 28636
MF_BPR: Epoch 1 of 1500. Elapsed time 0.67 sec
MF_BPR: Processed 30910 ( 99.99% ) in 0.75 seconds. BPR loss 4.77E-02. Sample per second: 41289
MF_BPR: Epoch 2 of 1500. Elapsed time 1.34 sec
MF_BPR: Processed 30910 ( 99.99% ) in 1.41 seconds. BPR loss 5.87E-02. Sample per second: 21965
MF_BPR: Epoch 3 of 1500. Elapsed time 2.00 sec
MF_BPR: Processed 30910 ( 99.99% ) in 1.07 seconds. BPR loss 6.95E-02. Sample per second: 28883
MF_BPR: Epoch 4 of 1500. Elapsed time 2.66 sec
MF_BPR: Processed 30910 ( 99.99% ) in 0.73 seconds. BPR loss 8.68E-02. Sample per second: 42405
MF_BPR: Validation begins...
EvaluatorHoldout: Processed 20381 ( 100.00% ) in 17.18 sec. Users per second: 1186
MF_BPR: CUTOFF: 10 - ROC_AUC: 0.0020280, PRECISION: 0.0004072, PRECISION_RECALL_MIN_DEN: 0.0010860, RECALL: 0.0010532, MAP: 0.0002994, MRR: 0.0012467, NDCG: 0.0006803, F1: 0.0005874, HIT_RATE: 0.0040724, ARHR: 0.0012467,